<a href="https://colab.research.google.com/github/Christy0216/ISE-540/blob/Final-Project/Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
pip install ijson

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 126 kB 4.7 MB/s 


In [ ]:
import ijson
import pandas as pd
import json
import csv
import numpy as np
import string
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
import re
import random
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier
from nltk.stem import LancasterStemmer
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
import imblearn
from collections import Counter
from imblearn.over_sampling import SMOTE
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import PCA
from sklearn.metrics import f1_score

#Load data

In [ ]:

lst = []
with open('/content/drive/MyDrive/Colab Notebooks/CAMFIRE_names_of_missing_found_people_internet_list.txt', newline='') as f:
    reader = csv.reader(f)
    data = list(reader)
for x in data:
    for i in x:
        lst.append(i.lower().split()) 

In [ ]:
filename = "/content/drive/MyDrive/Colab Notebooks/california_wildfire_2018_tweet_ids_4m.jsonl"
with open(filename, 'r') as f:
    objects = ijson.items(f, "full_text", multiple_values=True)
    columns = list(objects) 


In [ ]:
len(columns)

2276579

In [ ]:
# extract name of missing people
first_name = []
last_name = []
for i in range(len(lst)):
    if len(lst[i]) == 1:
        first_name.append(lst[i][0])
    else:
        first_name.append(lst[i][0])
        last_name.append(lst[i][-1]) 

In [ ]:
first_name = set(first_name)
last_name = set(last_name) 

In [ ]:
len(first_name)

3316

In [ ]:
len(last_name)

2246

In [ ]:
import functools
full_name=[]
name=list(functools.reduce(lambda x,y: x+y, data))
for i in name:
  full_name.append(i.lower())
full_name=set(full_name)

In [ ]:
text=[]
for i in columns:
  text.append(i.lower())

In [ ]:
len(full_name)

7415

In [ ]:
# tweets contains first name
use_txt1=[]
for i in first_name:
  for j in text:
    if i in j:
      use_txt1.append(j)

In [ ]:
use_txt1=set(use_txt1)

In [ ]:
#df=pd.DataFrame(use_txt1)
#df.to_csv('/content/drive/MyDrive/Colab Notebooks/use_txt.csv')

In [ ]:
# tweets contains last name
#use_txt2=[]
#for i in last_name:
#  for j in text:
#    if i in j:
#      use_txt1.append(j)

In [ ]:
#unique tweets contains name
len(set(use_txt1))

541995

In [ ]:
#load missing and found people name
found=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/hashtags_found_related.txt',header=None)
missing=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/hashtags_missing_lost_related.txt',header=None)

In [ ]:
fd_list=[]
for i in found[0]:
  fd_list.append(i.lower())

ms_list=[]
for j in missing[0]:
  ms_list.append(j.lower())

In [ ]:
label=set(use_txt1)

In [ ]:
label=list(label)

In [ ]:
#random sampling of 1000 valid tweets and manually label them
#sample=random.sample(label,1000)
#sample1=pd.DataFrame(sample)
#sample1.to_csv('/content/drive/MyDrive/Colab Notebooks/sample.csv')

In [ ]:
# tweets with missing hashtags
fd_txt=[]
for i in fd_list:
  for j in label:
    if i in j:
      fd_txt.append(j)

In [ ]:
# tweets with found hashtags
ms_txt=[]
for i in ms_list:
  for j in label:
    if i in j:
      ms_txt.append(j)

In [ ]:
fd=pd.DataFrame(fd_txt)
fd['y']=0
ms=pd.DataFrame(ms_txt)
ms['y']=1
fdms=pd.concat([fd,ms])
fdms.columns=['x','y']
#fdms.to_csv('/content/drive/MyDrive/Colab Notebooks/fdms.csv')

In [ ]:
Counter(fdms['y'])

Counter({0: 101, 1: 1114})

# Preprocessing

In [ ]:
# lowercase and remove punctuation, number, stop words, replace similar words with affix
def file_dict(file):
    test=[]
    for i in list(file['x']):
        index=list(file['x']).index(i)
        text1=str.maketrans({key:None for key in string.punctuation})
        t=i.translate(text1)
        t2=t.translate(str.maketrans('','',string.digits))
        t2=t2.lower()
        t2=t2.split()
        stop_words = ENGLISH_STOP_WORDS
        filtered_sentence = []
        lancaster = LancasterStemmer()
        for w in t2:
            if w not in stop_words and w.isalpha()==True and 'http' not in w:
                filtered_sentence.append((lancaster.stem(w)))
        test.append(filtered_sentence)
    total = []
    for lst in test:
      line = ''
      line += ' '.join(lst)
      total.append(line)
    return total

In [ ]:
# encode tweets with tfidf
def vectorizer(file):
  vectorizer=TfidfVectorizer(max_features=10000)
  X=vectorizer.fit_transform(file)
  return X

In [ ]:
# normalize x
def scale(file):
  scaler = StandardScaler()
  scaler.fit_transform(file)
  return file

In [ ]:
def pca(file):
  model = PCA(n_components=800,random_state=0)
  reduced = model.fit(file).transform(file)
  return reduced

In [ ]:
# random sampling 100k valid tweets for train&test
train=random.sample(label,100000)
train=pd.DataFrame(train)
train.columns=['x']

In [ ]:
# split train 80% /test 20% set
X_train_set, X_test_set= train_test_split(train, test_size=0.2, random_state=42)

,x,y
1306,subsidized dreams of escaping cities’\nundiffe...,1
1307,"@bethenny the people of paradise, ca need your...",1
1308,subsidized dreams of escaping cities’\nundiffe...,1
1309,@fema @realdonaldtrump @abc @nbcnews @cnn @msn...,1
1310,#campfire simply unbelievable. \ntriple initia...,1
...,...,...
1501,"""california wildfires lost &amp; found pets"" o...",1
1502,https://t.co/7htowmne3j\nthese are pictures of...,1
1503,"#campfire, #lostanimals do they look familiar?...",1
1504,"""unclaimed pets from the camp fire"" is a fb pa...",1


In [ ]:
#load train data 
train1200=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/sample - sample (2).csv',header=None)
train1200.columns=['x','y']
train1200_set=file_dict(train1200)
train1200_clean=vectorizer(train1200_set)

In [ ]:
X_train=pd.concat([X_train_set,train1200.iloc[-200:,:]])


In [ ]:
train_set=file_dict(X_train)

In [ ]:
#df1=pd.DataFrame(train_set)
#df1.to_csv('/content/drive/MyDrive/Colab Notebooks/train_set.csv')

In [ ]:
train_clean=vectorizer(train_set)
X_train=train_clean.toarray()
#test_clean=vectorizer(X_test_set)
#X_test=test_clean.toarray()
X_train=scale(X_train)
#X_test=scale(X_test)

In [ ]:
X_train=pca(X_train)

In [ ]:
#test_df=train.loc[X_test_index,]
#test600=test_df.sample(n=600,random_state=0)
#test600.to_csv('/content/drive/MyDrive/Colab Notebooks/test600.csv')

In [ ]:
x_1200=scale(train1200_clean.toarray())
y_1200=train1200['y']
y_1200=list(y_1200.astype('object'))
x_1200=pca(x_1200)

In [ ]:
len(y_1200)

1506

#Random Forest

In [ ]:
rfc = RandomForestClassifier()
parameters = {'n_estimators':[i for i in np.arange(100,201,50)],'max_depth': [j for j in np.arange(100,501,100)], 'random_state': [0]}
grid_rfc = GridSearchCV(rfc, parameters,cv=5,scoring='roc_auc')
grid_rfc = grid_rfc.fit(x_1200, y_1200)
grid_rfc.best_params_

{'max_depth': 100, 'n_estimators': 200, 'random_state': 0}

In [ ]:
grid_rfc.score

0.8742080676821764

In [ ]:
(1039+457)/1506

0.9933598937583001

In [ ]:
y_rf=grid_rfc.predict(X_train)

In [ ]:
y_pred=grid_rfc.predict(x_1200)
confusion_matrix(y_1200,y_pred)

array([[1039,    2],
       [   8,  457]])

#Logistic Regression

In [ ]:
 from sklearn.metrics import roc_auc_score

In [ ]:
lgr = LogisticRegression(random_state=0,class_weight='balanced')
lgr = lgr.fit(x_1200, y_1200)
y_lgr=lgr.predict(X_train)

In [ ]:
plgr=lgr.predict(x_1200)
roc_auc_score(y_1200,plgr)


0.9394254903783582

In [ ]:
y_pred=lgr.predict(x_1200)
confusion_matrix(y_1200,y_pred)

array([[991,  50],
       [ 34, 431]])

In [ ]:
(991+431)/1506

0.9442231075697212

#Decision Tree

In [ ]:
D = tree.DecisionTreeClassifier()
parameters = {'min_samples_leaf': [i for i in np.arange(2,11,1)],'max_depth': [j for j in np.arange(100,501,100)], 'random_state': [0]}
grid_dt = GridSearchCV(D, parameters,cv=5,scoring='roc_auc')
grid_dt.fit(x_1200, y_1200)

print(grid_dt.best_score_)

0.771426126222391


In [ ]:
grid_dt.best_params_

{'max_depth': 100, 'min_samples_leaf': 7, 'random_state': 0}

In [ ]:
y_dt=grid_dt.predict(X_train)
y_pred=grid_dt.predict(x_1200)
confusion_matrix(y_1200,y_pred)

array([[1004,   37],
       [  49,  416]])

In [ ]:
(1004+416)/1506

0.9428950863213812

#GradientBoostingClassifier

In [ ]:
#params = {'n_estimators': [100, 150, 200, 250, 300],
#          'learning_rate': np.arange(0.01, 0.1, 0.01),
#          'max_depth': [500,600,700,800,1000]}
#boost = GradientBoostingClassifier(random_state=20, max_features='log2')
#grid_boost = GridSearchCV(boost, param_grid = params, scoring='roc_auc')
#grid_boost.fit(x_1200, y_1200)

In [ ]:
#grid_boost.best_params_

In [ ]:
#grid_boost.best_score_

In [ ]:
boost = GradientBoostingClassifier(n_estimators=300,
                                   learning_rate=0.05,max_depth=500,random_state=20, max_features='log2')
boost.fit(x_1200, y_1200)

GradientBoostingClassifier(learning_rate=0.05, max_depth=500,
                           max_features='log2', n_estimators=300,
                           random_state=20)

In [ ]:
y_gbc=boost.predict(X_train)
pgbc=boost.predict(x_1200)
roc_auc_score(y_1200,pgbc)

0.9898422732484272

In [ ]:
y_pred=boost.predict(x_1200)
confusion_matrix(y_1200,y_pred)

array([[1040,    1],
       [   9,  456]])

In [ ]:
(1040+456)/1506

0.9933598937583001

#NaiveBayes

In [ ]:
B = BernoulliNB()
parameters = {'alpha': [i for i in np.arange(0.1,1,0.1)]}
grid_nb = GridSearchCV(B, parameters, cv=5, scoring='roc_auc')
grid_nb = grid_nb.fit(x_1200, y_1200)
grid_nb.best_params_

{'alpha': 0.1}

In [ ]:
print(grid_nb.best_score_)

0.7507240601786442


In [ ]:
y_nb=grid_nb.predict(X_train)

In [ ]:
y_pred=grid_nb.predict(x_1200)
confusion_matrix(y_1200,y_pred)

array([[956,  85],
       [ 89, 376]])

In [ ]:
(956+376)/1506

0.8844621513944223

#SGDClassifier

In [ ]:
params = {'alpha':np.arange(0.01,1,0.1)}
sgdc = SGDClassifier(class_weight='balanced')
grid_sgdc = GridSearchCV(sgdc, params, cv=5, scoring='roc_auc')
grid_sgdc = grid_sgdc.fit(x_1200, y_1200)
grid_sgdc.best_params_

{'alpha': 0.01}

In [ ]:
grid_sgdc.best_score_

0.8591831449535183

In [ ]:
y_sgdc=grid_sgdc.predict(X_train)
y_pred=grid_sgdc.predict(x_1200)
confusion_matrix(y_1200,y_pred)

array([[997,  44],
       [148, 317]])

In [ ]:
(997+317)/1506

0.8725099601593626

#XGBoost

In [ ]:
#params = {'n_estimators': [400,500,600],
#          'max_depth': [300,500,700],
#          'learning_rate': np.arange(0.01, 0.05, 0.01)}
#xgboost = XGBClassifier()
#grid_xgboost = GridSearchCV(xgboost, param_grid=params, scoring='roc_auc')
#grid_xgboost.fit(x_1200, y_1200)
#grid_xgboost.best_params_

In [ ]:
#grid_xgboost.best_score_

In [ ]:
xgboost = XGBClassifier(n_estimators=600,max_depth=300,learning_rate=0.01)
xgboost.fit(x_1200, y_1200)

XGBClassifier(learning_rate=0.01, max_depth=300, n_estimators=600)

In [ ]:
y_xgb=xgboost.predict(X_train)
y_pred=xgboost.predict(x_1200)
confusion_matrix(y_1200,y_pred)

array([[1037,    4],
       [   6,  459]])

In [ ]:
pxgb=xgboost.predict(x_1200)
roc_auc_score(y_1200,pxgb)

0.9916271575098387

In [ ]:
(1037+459)/1506

0.9933598937583001

#SVC

In [ ]:
params = {'C': [0.1,1, 10, 100],
          'gamma': [1,0.1,0.01,0.001],
          'kernel': ['rbf', 'poly', 'sigmoid']}
svc = SVC(random_state=0,class_weight='balanced')
grid_svc = GridSearchCV(svc, param_grid=params, scoring='roc_auc')
grid_svc.fit(x_1200, y_1200)
grid_svc.best_params_

{'C': 0.1, 'gamma': 1, 'kernel': 'sigmoid'}

In [ ]:
grid_svc.best_score_

0.8604943337647073

In [ ]:
y_svc=grid_svc.predict(X_train)

In [ ]:
y_pred=grid_svc.predict(x_1200)
confusion_matrix(y_1200,y_pred)

array([[992,  49],
       [113, 352]])

In [ ]:
(992+352)/1506

0.8924302788844621

#Combine y_pred

In [ ]:
rf_y=pd.DataFrame(y_rf)
lgr_y=pd.DataFrame(y_lgr)
dt_y=pd.DataFrame(y_dt)
gbc_y=pd.DataFrame(y_gbc)
nb_y=pd.DataFrame(y_nb)
sgdc_y=pd.DataFrame(y_sgdc)
xgb_y=pd.DataFrame(y_xgb)
svc_y=pd.DataFrame(y_svc)
y_pred=pd.concat([rf_y,lgr_y,dt_y,gbc_y,nb_y,sgdc_y,xgb_y,svc_y,],axis=1)
y_pred.columns=['rf','lgr','dt','gbc','nb','sgdc','xgb','svc']
y_pred

,rf,lgr,dt,gbc,nb,sgdc,xgb,svc
0,0,0,0,0,0,0,0,0
1,0,0,1,0,0,0,1,0
2,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,1,0
4,0,1,1,0,1,1,1,1
...,...,...,...,...,...,...,...,...
80195,0,0,0,0,1,0,0,0
80196,0,0,0,0,0,0,0,0
80197,0,1,0,0,0,0,0,0
80198,0,0,0,0,0,0,0,0


In [ ]:
y_pred['sum']=y_pred.sum(axis = 1)

In [ ]:
y_pred['y']=np.nan

In [ ]:
y_pred[y_pred['sum']>=3]

,rf,lgr,dt,gbc,nb,sgdc,xgb,svc,sum,y
4,0,1,1,0,1,1,1,1,6,NaN
8,0,1,1,0,0,1,1,1,5,NaN
9,0,1,0,0,0,1,0,1,3,NaN
13,0,1,1,0,1,0,0,0,3,NaN
16,0,1,1,0,1,1,0,1,5,NaN
...,...,...,...,...,...,...,...,...,...,...
80089,0,1,0,0,0,1,0,1,3,NaN
80092,0,0,1,0,1,0,1,0,3,NaN
80122,0,1,0,0,1,0,0,1,3,NaN
80146,0,1,0,0,1,0,0,1,3,NaN


In [ ]:
# if 4 classifier predict 1, label 1, else 0
for i in range(len(X_train)):
  if y_pred['sum'][i]>=3:
    y_pred['y'][i]=1
  else:
    y_pred['y'][i]=0

y_pred

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,rf,lgr,dt,gbc,nb,sgdc,xgb,svc,sum,y
0,0,0,0,0,0,0,0,0,0,0.0
1,0,0,1,0,0,0,1,0,2,0.0
2,0,0,0,0,0,0,0,0,0,0.0
3,0,1,0,0,0,0,1,0,2,0.0
4,0,1,1,0,1,1,1,1,6,1.0
...,...,...,...,...,...,...,...,...,...,...
80195,0,0,0,0,1,0,0,0,1,0.0
80196,0,0,0,0,0,0,0,0,0,0.0
80197,0,1,0,0,0,0,0,0,1,0.0
80198,0,0,0,0,0,0,0,0,0,0.0


In [ ]:
Counter(y_pred['y'])

Counter({0.0: 65825, 1.0: 14375})

In [ ]:
y_train=list(y_pred['y'].astype('object'))


#Use labeled train set to train again

In [ ]:
# manually labeled 600 random sample from test set and compare with predicted label
test_600=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/test600 - test600 (2).csv')
test_600

,Unnamed: 0,x,y
0,89114.0,seriously? https://t.co/hxcirgbkbr,0
1,44259.0,"heartbroken. | death toll rises to 23 as 200,...",0
2,62841.0,me 10 years ago: “this sounds like hyperbole”\...,0
3,20284.0,california is on fire. it won't be the last ti...,0
4,36330.0,the camp fire currently burning in northern ca...,1
...,...,...,...
795,NaN,rt @2h4pfoundation: looking for a pet that you...,1
796,NaN,wildfires sweeping #california have left more ...,1
797,NaN,here is the actual list of names of #missingpe...,1
798,NaN,please respond found or any updates on people/...,1


In [ ]:
test600=vectorizer(file_dict(test_600))
x_600=scale(test600.toarray())
x_600 = pca(x_600)
y_600=test_600['y']
y_600=list(y_600.astype('object'))

In [ ]:
Counter(y_600)

Counter({0: 627, 1: 173})

In [ ]:
# random forest
rfc = RandomForestClassifier(max_depth= 100, n_estimators= 200, random_state= 0)
rfc.fit(X_train,y_train)


RandomForestClassifier(max_depth=100, n_estimators=200, random_state=0)

In [ ]:
rfc_600=rfc.predict(x_600)
confusion_matrix(y_600,rfc_600)

array([[627,   0],
       [173,   0]])

In [ ]:
rf_f1=f1_score(y_600,rfc_600)

In [ ]:
# logistic regression
lgr = LogisticRegression(random_state=0,class_weight='balanced')
lgr.fit(X_train,y_train)

LogisticRegression(class_weight='balanced', random_state=0)

In [ ]:
lgr_600=lgr.predict(x_600)
confusion_matrix(y_600,lgr_600)

array([[479, 148],
       [ 84,  89]])

In [ ]:
lgr_f1=f1_score(y_600,lgr_600)

In [ ]:
# decision tree
dt = tree.DecisionTreeClassifier(max_depth= 100, min_samples_leaf= 7, random_state= 0)
dt.fit(X_train,y_train)

DecisionTreeClassifier(max_depth=100, min_samples_leaf=7, random_state=0)

In [ ]:
dt_600=dt.predict(x_600)
confusion_matrix(y_600,dt_600)

array([[511, 116],
       [ 99,  74]])

In [ ]:
dt_f1=f1_score(y_600,dt_600)

In [ ]:
# gradient boosting classifier
boost = GradientBoostingClassifier(n_estimators=100,
                                   learning_rate=0.08, max_depth=500, random_state=20, max_features='log2')
boost.fit(X_train,y_train)

GradientBoostingClassifier(learning_rate=0.08, max_depth=500,
                           max_features='log2', random_state=20)

In [ ]:
boost_600=boost.predict(x_600)
confusion_matrix(y_600,boost_600)

array([[627,   0],
       [173,   0]])

In [ ]:
boost_f1=f1_score(y_600,boost_600)

In [ ]:
# naive bayes
nb = BernoulliNB(alpha=0.1)
nb.fit(X_train,y_train)

BernoulliNB(alpha=0.1)

In [ ]:
nb_600=nb.predict(x_600)
confusion_matrix(y_600,nb_600)

array([[623,   4],
       [168,   5]])

In [ ]:
nb_f1=f1_score(y_600,nb_600)

In [ ]:
# sgdc classifier
sgdc = SGDClassifier(class_weight='balanced',alpha=0.1)
sgdc.fit(X_train,y_train)

SGDClassifier(alpha=0.1, class_weight='balanced')

In [ ]:
sgdc_600=sgdc.predict(x_600)
confusion_matrix(y_600,sgdc_600)

array([[238, 389],
       [ 35, 138]])

In [ ]:
sgdc_f1=f1_score(y_600,sgdc_600)

In [ ]:
# xgboost
#xgboost = XGBClassifier(n_estimators=600,max_depth=300,learning_rate=0.01)
#xgboost.fit(X_train,y_train)

In [ ]:
#xgboost_600=xgboost.predict(x_600)
#confusion_matrix(y_600,xgboost_600)

In [ ]:
# svc
#svc = SVC(random_state=0,class_weight='balanced',C= 0.1, gamma= 1, kernel= 'rbf')
#svc.fit(X_train,y_train)

In [ ]:
#svc_600=svc.predict(x_600)
#confusion_matrix(y_600,svc_600)

In [ ]:
rfcy=pd.DataFrame(rfc_600)
lgry=pd.DataFrame(lgr_600)
dty=pd.DataFrame(dt_600)
gbcy=pd.DataFrame(boost_600)
nby=pd.DataFrame(nb_600)
sgdcy=pd.DataFrame(sgdc_600)
pred_y=pd.concat([rfcy,lgry,dty,gbcy,nby,sgdcy],axis=1)
pred_y.columns=['rf','lgr','dt','gbc','nb','sgdc']
pred_y

,rf,lgr,dt,gbc,nb,sgdc
0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,1.0,1.0,0.0,0.0,1.0
...,...,...,...,...,...,...
795,0.0,0.0,1.0,0.0,0.0,1.0
796,0.0,1.0,0.0,0.0,0.0,1.0
797,0.0,1.0,1.0,0.0,0.0,1.0
798,0.0,1.0,0.0,0.0,0.0,1.0


In [ ]:
pred_y['sum']=pred_y.sum(axis = 1)
pred_y['y']=np.nan
pred_y[pred_y['sum']>=2]

,rf,lgr,dt,gbc,nb,sgdc,sum,y
2,0.0,0.0,1.0,0.0,0.0,1.0,2.0,NaN
4,0.0,1.0,1.0,0.0,0.0,1.0,3.0,NaN
7,0.0,0.0,1.0,0.0,0.0,1.0,2.0,NaN
9,0.0,1.0,0.0,0.0,0.0,1.0,2.0,NaN
11,0.0,1.0,0.0,0.0,0.0,1.0,2.0,NaN
...,...,...,...,...,...,...,...,...
795,0.0,0.0,1.0,0.0,0.0,1.0,2.0,NaN
796,0.0,1.0,0.0,0.0,0.0,1.0,2.0,NaN
797,0.0,1.0,1.0,0.0,0.0,1.0,3.0,NaN
798,0.0,1.0,0.0,0.0,0.0,1.0,2.0,NaN


In [ ]:
for i in range(800):
  if pred_y['sum'][i]>=2:
    pred_y['y'][i]=1
  else:
    pred_y['y'][i]=0

pred_y

,rf,lgr,dt,gbc,nb,sgdc,sum,y
0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,1.0,2.0,1.0
3,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
4,0.0,1.0,1.0,0.0,0.0,1.0,3.0,1.0
...,...,...,...,...,...,...,...,...
795,0.0,0.0,1.0,0.0,0.0,1.0,2.0,1.0
796,0.0,1.0,0.0,0.0,0.0,1.0,2.0,1.0
797,0.0,1.0,1.0,0.0,0.0,1.0,3.0,1.0
798,0.0,1.0,0.0,0.0,0.0,1.0,2.0,1.0


In [ ]:
confusion_matrix(y_600,pred_y['y'])

array([[434, 193],
       [ 61, 112]])

In [ ]:
f1_score(y_600,pred_y['y'])

0.4686192468619247

In [ ]:
list1=[]
list1.append(rf_f1)
list1.append(lgr_f1)
list1.append(dt_f1)
list1.append(nb_f1)
list1.append(boost_f1)
list1.append(sgdc_f1)
f1=pd.DataFrame(list1).T
f1.columns=['random forest','logistic regression','decision tree','naive bayes','gradient boost','sgdc']
f1

,random forest,logistic regression,decision tree,naive bayes,gradient boost,sgdc
0,0.0,0.434146,0.407713,0.054945,0.0,0.394286
